In [1]:
# Importation of librairies and functions
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import json
from os.path import join

In [2]:
os.chdir("C:\\Users\\Juliette\\Research\\Projects\\ReSync")

In [3]:
project_path = os.getcwd()
print(project_path)


# import custom-made functions:
from utils import *
from resync_function import run_resync
from interactive import *
from tmsi_poly5reader import *
from scripts.loading_data import _load_mat_file, _load_data_lfp, _load_TMSi_artefact_channel, _load_intracranial_csv_file, _load_external_csv_file

C:\Users\Juliette\Research\Projects\ReSync


In [4]:
def replace_with_shuffled_values(signal, replace_start, replace_end, shuffle_start, shuffle_end):
    # Extract values from the specified part of the middle
    selected_values = signal[shuffle_start:shuffle_end]

    # Shuffle the selected values
    shuffled_values = np.random.permutation(selected_values)

    # Calculate the number of repetitions needed
    repetitions = (replace_end - replace_start) // len(shuffled_values)

    # Pad the shuffled values if needed
    shuffled_values_padded = np.tile(shuffled_values, repetitions + 1)[:replace_end - replace_start]

    # Replace the specified part of the signal with shuffled values
    randomized_signal = np.copy(signal)
    randomized_signal[replace_start:replace_end] = shuffled_values_padded

    return randomized_signal

## INTRACRANIAL ##

In [5]:
os.chdir(project_path)

In [6]:
os.getcwd()

'C:\\Users\\Juliette\\Research\\Projects\\ReSync'

In [7]:
sub_ID = 'sub084 3MFU M1S0 randomized'
fname_lfp = 'sub-20230905PStn_ses-2023121802263096_run-BrainSense20231218024800.mat'
ch_idx_lfp = 0

In [8]:
#  Set saving path
saving_path = join("results", sub_ID)
if not os.path.isdir(saving_path):
    os.makedirs(saving_path)

#  Set source path
source_path = "sourcedata"

In [9]:
dataset_lfp= _load_mat_file(sub_ID, fname_lfp, saving_path)
LFP_array, lfp_sig, LFP_rec_ch_names, sf_LFP = _load_data_lfp(sub_ID, dataset_lfp, 
                                                        ch_idx_lfp, saving_path)

Creating RawArray with float64 data, n_channels=6, n_times=74375
    Range : 0 ... 74374 =      0.000 ...   297.496 secs
Ready.


c:\Users\Juliette\anaconda3\envs\resync\Lib\site-packages\pymatreader\utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
C:\Users\Juliette\Research\Projects\ReSync\scripts\loading_data.py:57: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  data = read_raw_fieldtrip(
C:\Users\Juliette\Research\Projects\ReSync\scripts\loading_data.py:57: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_02. Making it a MISC channel.
  data = read_raw_fieldtrip(
C:\Users\Juliette\Research\Projects\ReSync\scripts\loading_data.py:57: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_R_13. Making it a MISC channel.
  data = read_raw_fieldtrip(
C:\Users\Juliette\Research\Projects\ReSync\scripts\loading_data.

In [10]:
lfp_sig.shape

(74375,)

In [11]:
%matplotlib qt
plt.plot(lfp_sig)

In [12]:
signal1 =  np.copy(lfp_sig)

# Specify the start and end indices for the middle part to be replaced
middle_replace_start = 3900
middle_replace_end = 71680

# Specify the start and end indices for the part to be used for shuffling
shuffle_start = 3900
shuffle_end = 71680

# Replace the specified part of the signal with shuffled values
randomized_signal1 = replace_with_shuffled_values(signal1, middle_replace_start, middle_replace_end, shuffle_start, shuffle_end)

# Plot original and randomized signals
plt.figure(figsize=(10, 6))
#plt.plot(lfp_sig, label='Original Signal')
plt.plot(randomized_signal1, label='Randomized Signal')
plt.title('Signal Randomization Example')
plt.legend()
plt.show()


In [14]:
# now to shuffle beginning of the signal:
signal1 =  np.copy(randomized_signal1)

# Specify the start and end indices for the part to be replaced
beginning_replace_start = 0
beginning_replace_end = 1980

# Specify the start and end indices for the part to be used for shuffling
beginning_shuffle_start = 0
beginning_shuffle_end = 1980

# Replace the specified part of the signal with shuffled values
randomized_signal_lfp = replace_with_shuffled_values(signal1, beginning_replace_start, beginning_replace_end, beginning_shuffle_start, beginning_shuffle_end)

# Plot original and randomized signals
plt.figure(figsize=(10, 6))
#plt.plot(lfp_sig, label='Original Signal')
plt.plot(randomized_signal_lfp, label='Randomized Signal')
plt.title('Signal Randomization Example')
plt.legend()
plt.show()


In [15]:
randomized_signal_lfp_df = pd.DataFrame(randomized_signal_lfp, columns=["LFP_Stn_L_02"])

In [16]:
randomized_signal_lfp_df

,LFP_Stn_L_02
0,-1.565633
1,8.163657
2,14.202526
3,25.273786
4,-15.209004
...,...
74370,25.385617
74371,25.721110
74372,23.037168
74373,19.682241


In [17]:
# save as csv:
randomized_signal_lfp_df.to_csv(saving_path + '\\Intracerebral_LFP_' 
                                      + str(sub_ID) + '_' + str(sf_LFP) + 'Hz.csv', index=False)

## EXTERNAL ##

In [18]:
#source_path = "sourcedata"
fname_external = 'sub084_3mfu_m1s0_BrStr_restTap - 20231218T154811.DATA.Poly5'
BIP_ch_name = 'Bip25'

##  External
#source_path = join(main_directory,'sourcedata')
TMSi_data = Poly5Reader(join(source_path, fname_external)) 
(BIP_channel, external_file, external_rec_ch_names, 
sf_external, ch_index_external)= _load_TMSi_artefact_channel(sub_ID, TMSi_data, 
                                                            fname_external, BIP_ch_name, saving_path)


Reading file  sourcedata\sub084_3mfu_m1s0_BrStr_restTap - 20231218T154811.DATA.Poly5
	 Number of samples:  729847 
	 Number of channels:  38 
	 Sample rate: 2048 Hz
Done reading data.
Creating RawArray with float64 data, n_channels=38, n_times=729847
    Range : 0 ... 729846 =      0.000 ...   356.370 secs
Ready.


In [19]:
plt.plot(BIP_channel)

In [20]:
from utils import _filtering

In [21]:
BIP_channel_filtered = _filtering(BIP_channel)

In [22]:
plt.plot(BIP_channel_filtered)

In [24]:
signal1 =  np.copy(BIP_channel_filtered)

# Specify the start and end indices for the middle part to be replaced
middle_replace_start = 71360
middle_replace_end = 628480

# Specify the start and end indices for the part to be used for shuffling
shuffle_start = 71360
shuffle_end = 628480

# Replace the specified part of the signal with shuffled values
randomized_signal1 = replace_with_shuffled_values(signal1, middle_replace_start, middle_replace_end, shuffle_start, shuffle_end)

# Plot original and randomized signals
plt.figure(figsize=(10, 6))
plt.plot(BIP_channel, label='Original Signal')
plt.plot(randomized_signal1, label='Randomized Signal')
plt.title('Signal Randomization Example')
plt.legend()
plt.show()


In [25]:

# now to shuffle beginning of the signal:
signal1 =  np.copy(randomized_signal1)

# Specify the start and end indices for the part to be replaced
beginning_replace_start = 0
beginning_replace_end = 57250

# Specify the start and end indices for the part to be used for shuffling
beginning_shuffle_start = 0
beginning_shuffle_end = 57250

# Replace the specified part of the signal with shuffled values
randomized_signal2 = replace_with_shuffled_values(signal1, beginning_replace_start, beginning_replace_end, beginning_shuffle_start, beginning_shuffle_end)

# Plot original and randomized signals
plt.figure(figsize=(10, 6))
plt.plot(BIP_channel, label='Original Signal')
plt.plot(randomized_signal2, label='Randomized Signal')
plt.title('Signal Randomization Example')
plt.legend()
plt.show()

In [26]:
randomized_signal2 = pd.DataFrame(randomized_signal2, columns=["Bip25"])

In [27]:
randomized_signal2

,Bip25
0,-0.697413
1,-10.889383
2,-10.138157
3,-5.485416
4,-27.815863
...,...
729842,-244.567220
729843,-205.428299
729844,-171.410757
729845,-141.666528


In [28]:
randomized_signal2.to_csv(saving_path + '\\External_data_' 
                                      + str(sub_ID) + '_' + str(sf_external) + 'Hz.csv', index=False)